<a href="https://colab.research.google.com/github/LiuYuWei/Taiwan-Health-MCP/blob/fix%2Fgemini-compatibility-and-init/taiwan-health-mcp-sse-server-workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Clone 專案
!git clone -b fix/gemini-compatibility-and-init https://github.com/LiuYuWei/Taiwan-Health-MCP.git

Cloning into 'Taiwan-Health-MCP'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 235 (delta 85), reused 201 (delta 59), pack-reused 0 (from 0)
Receiving objects: 100% (235/235), 6.09 MiB | 23.46 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
%cd /content/Taiwan-Health-MCP

!pip install -r requirements.txt


/content/Taiwan-Health-MCP
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: mcp
    Found existing installation: mcp 1.24.0
    Uninstalling mcp-1.24.0:
      Successfully uninstalled mcp-1.24.0
ERROR: pip's dependency resolver does no

In [5]:
!python src/server.py

[INFO][2026-01-10 01:23:33] Using DATA_DIR: /content/Taiwan-Health-MCP/data
[INFO][2026-01-10 01:23:33] Found ICD Excel file: /content/Taiwan-Health-MCP/data/1.2023年中文版ICD-10-CM_PCS_1131118V3(修改ICD-10-CM之N80.A0等中文名稱).xlsx
[INFO][2026-01-10 01:23:33] Initializing Services...
[INFO][2026-01-10 01:23:33] ICD Database found at: /content/Taiwan-Health-MCP/data/icd10_smart.db
[01/10/26 01:23:33] INFO     Adding job tentatively -- it will be    ]8;id=646989;file:///usr/local/lib/python3.12/dist-packages/apscheduler/schedulers/base.py\base.py]8;;\:]8;id=611050;file:///usr/local/lib/python3.12/dist-packages/apscheduler/schedulers/base.py#507\507]8;;\
                             properly scheduled when the scheduler              
                             starts                                             
                    INFO     Added job                              ]8;id=578619;file:///usr/local/lib/python3.12/dist-packages/apscheduler/schedulers/base.py\base.py]8;;\:]8;

In [6]:
!pip install pyngrok

  Using cached pyngrok-7.5.0-py3-none-any.whl.metadata (8.1 kB)
Using cached pyngrok-7.5.0-py3-none-any.whl (24 kB)


In [7]:
import os
import subprocess
import threading
import time
import json
import sys
from pyngrok import ngrok

# --- 第二部分：設定與啟動 ---
# 請在此處填入您的 Ngrok Authtoken
from google.colab import userdata
NGROK_TOKEN = userdata.get('NGROK_TOKEN')


if NGROK_TOKEN == "在此貼上你的_NGROK_AUTHTOKEN":
    print("\n❌ 錯誤：請先在程式碼中設定 NGROK_TOKEN 才能開啟外部連線！")
    print("註冊網址: https://dashboard.ngrok.com/get-started/your-authtoken")
else:
    # 1. 設定 Ngrok 隧道
    ngrok.set_auth_token(NGROK_TOKEN)

    try:
        public_url = ngrok.connect(8000).public_url
        sse_url = f"{public_url}/sse"
    except Exception as e:
        print(f"❌ Ngrok 連線失敗: {e}")
        sys.exit(1)

    # 2. 定義即時輸出日誌的啟動函式
    def run_server_with_logs():
        print(f"\n📦 正在背景啟動 MCP Server 並進行資料初始化 (ETL)...")
        print(f"⚠️  第一次啟動會下載 FDA 數萬筆資料，請耐心候約 3-5 分鐘。\n")

        # 使用 subprocess 抓取 python 的即時輸出
        process = subprocess.Popen(
            ["python", "src/server.py", "sse"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )

        # 讀取每一行日誌並印出
        for line in iter(process.stdout.readline, ""):
            clean_line = line.strip()
            print(f"[Server] {clean_line}")

            # 偵測到關鍵字代表啟動完成
            if "Application startup complete" in clean_line or "Uvicorn running on" in clean_line:
                print("\n" + "🌟"*25)
                print("✅ MCP 伺服器已正式就緒！")
                print(f"🔗 Claude 連線網址: {sse_url}")
                print("🌟"*25 + "\n")

    # 3. 在背景執行
    mcp_thread = threading.Thread(target=run_server_with_logs, daemon=True)
    mcp_thread.start()

    # 4. 立即輸出 Claude 設定 JSON 供複製
    config_json = {
        "mcpServers": {
            "taiwan-health-colab": {
                "url": sse_url,
                "transport": "sse"
            }
        }
    }

    print("\n" + "="*60)
    print("📋 請將以下 JSON 內容複製到 Claude Desktop 設定中：")
    print("-" * 60)
    print(json.dumps(config_json, indent=2))
    print("-" * 60)
    print(f"💡 您的連線網址預約為: {sse_url}")
    print("⚠️  請觀察下方日誌，看到 'Application startup complete' 後再使用。")
    print("="*60 + "\n")


📦 正在背景啟動 MCP Server 並進行資料初始化 (ETL)...
⚠️  第一次啟動會下載 FDA 數萬筆資料，請耐心候約 3-5 分鐘。


📋 請將以下 JSON 內容複製到 Claude Desktop 設定中：
------------------------------------------------------------
{
  "mcpServers": {
    "taiwan-health-colab": {
      "url": "https://bf20e6d8b15c.ngrok-free.app/sse",
      "transport": "sse"
    }
  }
}
------------------------------------------------------------
💡 您的連線網址預約為: https://bf20e6d8b15c.ngrok-free.app/sse
⚠️  請觀察下方日誌，看到 'Application startup complete' 後再使用。

